# Live Plotting During Experiments

This notebook demonstrates real-time data visualization during scans.

## Features

- Live matplotlib plots
- Live plotly plots (interactive)
- Progress bars during scans

In [ ]:
from rust_daq import Motor, Detector
from rust_daq.jupyter import (
    quick_connect,
    LivePlot,
    live_scan_plot,
)
import numpy as np
import time

In [ ]:
# Connect to daemon
conn = quick_connect()
conn.__enter__()

In [ ]:
motor = Motor("mock_stage")
detector = Detector("mock_power_meter")

## Live Matplotlib Plot

Plot updates in real-time as data is acquired.

In [ ]:
%matplotlib notebook

data = live_scan_plot(
    motor=motor,
    detector=detector,
    start=0.0,
    stop=100.0,
    steps=30,
    dwell_time=0.05,
    backend='matplotlib'
)

## Live Plotly Plot (Interactive)

Plotly plots are fully interactive with zoom, pan, and hover.

In [ ]:
data_plotly = live_scan_plot(
    motor=motor,
    detector=detector,
    start=0.0,
    stop=100.0,
    steps=30,
    dwell_time=0.05,
    backend='plotly'
)

## Manual Live Plotting

For more control, create and update plots manually.

In [ ]:
%matplotlib notebook

plot = LivePlot(backend='matplotlib', title='Custom Scan')

positions = np.linspace(0, 100, 25)

for pos in positions:
    motor.position = pos
    time.sleep(0.05)
    value = detector.read()
    
    plot.update([pos], [value])

plot.finalize()

## Multi-Dataset Live Plotting

Plot multiple detectors simultaneously.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

plt.ion()
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

ax1.set_title('Detector 1')
ax1.set_xlabel('Position (mm)')
ax1.set_ylabel('Reading')
ax1.grid(True, alpha=0.3)

ax2.set_title('Position History')
ax2.set_xlabel('Step')
ax2.set_ylabel('Position (mm)')
ax2.grid(True, alpha=0.3)

line1, = ax1.plot([], [], 'o-', color='blue')
line2, = ax2.plot([], [], 's-', color='red')

plt.tight_layout()
plt.show()

# Data storage
x_data = []
y_data = []
steps = []
positions = np.linspace(0, 100, 20)

for i, pos in enumerate(positions):
    motor.position = pos
    time.sleep(0.05)
    value = detector.read()
    
    x_data.append(pos)
    y_data.append(value)
    steps.append(i)
    
    # Update plots
    line1.set_data(x_data, y_data)
    ax1.relim()
    ax1.autoscale_view()
    
    line2.set_data(steps, x_data)
    ax2.relim()
    ax2.autoscale_view()
    
    fig.canvas.draw()
    fig.canvas.flush_events()

plt.ioff()
plt.show()

## Export Data for Further Analysis

In [ ]:
import pandas as pd

# Convert to DataFrame
df = pd.DataFrame({
    'position': x_data,
    'reading': y_data
})

# Display summary statistics
print(df.describe())

# Save to CSV
# df.to_csv('scan_results.csv', index=False)

In [ ]:
# Cleanup
conn.__exit__(None, None, None)